In [9]:
# https://stackoverflow.com/questions/16515099/saving-a-stream-while-playing-it-using-libvlc
# vlc your_input_file_or_stream_here --sout="#std{access=file,mux=ps,dst=go.mpg}" 
import vlc
import os
import time
import numpy as np
import requests

from IPython.display import clear_output


In [10]:
#  0: 'NothingSpecial',
#  1: 'Opening',
#  2: 'Buffering',
#  3: 'Playing',
#  4: 'Paused',
#  5: 'Stopped',
#  6: 'Ended',
#  7: 'Error'

In [11]:
import vlc
import time
import numpy as np
import multiprocessing

class ExperimentClient():
    def __init__(self, url='http://131.234.250.116:9000/data/stream_1.m3u8'):
        stream_url = url
        instance = vlc.Instance("--vout=dummy --aout=dummy".split())
        
        self.media = instance.media_new(stream_url)
        self.player = instance.media_player_new()

        self.player.set_media(self.media)

        self.bitrates = []
        self.playing_time = 0

    def restart_player(self):
        self.player.stop()
        self.player.play()

    def start_player(self):
        self.player.play()

    def stop_player(self):
        self.player.stop()

    def consume_stream_proc(self, timeblock, return_dict):
        _counter = 0
        # return_dict = {
        #     "bitrates": [],
        #     "playing_time": 0
        # }
        _just_starting = True
        self.start_player()
        # init_time = time.time()
        previous_bitrate = 0.0
        _same_bitrate_count = 0
        SAME_BITRATE_THRESHOLD = 2
        time.sleep(1)
        while(_counter < timeblock+1):
            try:
                stats = vlc.MediaStats() 
                state = str(self.media.get_state())
                _bitrate = 0.0
                if state in ["State.Opening", "State.Buffering", "State.Playing"]:
                    self.media.get_stats(stats)
                    _bitrate = stats.demux_bitrate*8000.0

                    print("Bitrate")
                    print(_bitrate)
                    print(state)
                    if _bitrate == 0.0:
                        if _just_starting:
                            print("Just starting!")
                            _just_starting = False
                        else:
                            print("No data yet...")
                            return_dict['down_time'] += 1
                    else:
                        if _bitrate == previous_bitrate:
                            if _same_bitrate_count >= SAME_BITRATE_THRESHOLD:
                                _same_bitrate_count = 0
                                print("Same bitrate, Retrying...")
                                return_dict['down_time'] += SAME_BITRATE_THRESHOLD + 1
                                return_dict['playing_time'] -= SAME_BITRATE_THRESHOLD
                                self.restart_player()
                            else:
                                print("Same bitrate ++ !!")
                                _same_bitrate_count += 1
                                return_dict['bitrates'].append(_bitrate)
                                return_dict['playing_time'] += 1
                        else:
                            _same_bitrate_count = 0
                            return_dict['bitrates'].append(_bitrate)
                            return_dict['playing_time'] += 1

                else:
                    if _just_starting:
                        print("Just starting!")
                        _just_starting = False                    
                    else:
                        print("Retrying...")
                        return_dict['down_time'] += 1                    
                        self.restart_player()

            except Exception as e:
                print(e)

            print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
            print("\n\n")
            previous_bitrate = _bitrate
            time.sleep(1)
            _counter += 1

        self.stop_player()        
        return return_dict


In [12]:
MANO_URL = "thesismano1.cs.upb.de"
def set_remote_version(version, host=MANO_URL, port=8898) :
    if "virtual_deployment_units_vm" in version:
        switch_type="VM"
    if "virtual_deployment_units_gpu" in version:
        switch_type="GPU"
    if "virtual_deployment_units_con" in version:
        switch_type="CON"



    _base_path = 'http://{0}:{1}/switch_version?version={2}'.format(host, port, switch_type)

    try:
        r = requests.get(_base_path, verify=False)
        print("Switch Version")
        print(r.text)
    except Exception as e:
        print(e)
        print("Switch version could'nt be set")


In [13]:
from IPython.display import clear_output
# clear_output(wait=True)

manager = multiprocessing.Manager()
return_dict = manager.dict()

return_dict["bitrates"] = manager.list()
return_dict["playing_time"] = 0
return_dict["down_time"] = 0

_runner = ExperimentClient()

# output = _runner.consume_stream_proc(60, {})
timeblock = 120
p = multiprocessing.Process(target=_runner.consume_stream_proc, args=(timeblock, return_dict))
p.start()
p_killed = False

while(p.is_alive()):
    # print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
    # print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
    # clear_output(wait=True)
    time.sleep(1)

print("\n####################")
print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
print("####################\n")

p.terminate()
set_remote_version("virtual_deployment_units_con:transcoder-image-1-con")
# set_remote_version("virtual_deployment_units_gpu:transcoder-image-1-gpu")

return_dict = manager.dict()

return_dict["bitrates"] = manager.list()
return_dict["playing_time"] = 0
return_dict["down_time"] = 0

p = multiprocessing.Process(target=_runner.consume_stream_proc, args=(timeblock, return_dict))
p.start()

while(p.is_alive()):
    # print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
    # print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
    # clear_output(wait=True)
    time.sleep(1)

print("\n####################")
print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
print("####################\n")

#     if req_serv_id in self.active_services:
#         # LOG.info("Still active..." + req_serv_id)
#         pass
#     else:
#         LOG.info("Killing training..." + req_serv_id)
#         p.terminate()
#         p_killed = True




Bitrate
0.0
State.Playing
Just starting!
[] 0 0



Bitrate
107.96960443258286
State.Playing
[107.96960443258286] 1 0



Bitrate
796.8419194221497
State.Playing
[107.96960443258286, 796.8419194221497] 2 0



Bitrate
746.8152642250061
State.Playing
[107.96960443258286, 796.8419194221497, 746.8152642250061] 3 0



Bitrate
746.5298771858215
State.Playing
[107.96960443258286, 796.8419194221497, 746.8152642250061, 746.5298771858215] 4 0



Bitrate
746.414303779602
State.Playing
[107.96960443258286, 796.8419194221497, 746.8152642250061, 746.5298771858215, 746.414303779602] 5 0



Bitrate
740.3742074966431
State.Playing
[796.8419194221497, 746.8152642250061, 746.5298771858215, 746.414303779602, 740.3742074966431] 6 0



Bitrate
741.3706183433533
State.Playing
[746.8152642250061, 746.5298771858215, 746.414303779602, 740.3742074966431, 741.3706183433533] 7 0



Bitrate
746.1509108543396
State.Playing
[746.5298771858215, 746.414303779602, 740.3742074966431, 741.3706183433533, 746.1509108543396] 8




Bitrate
727.6479005813599
State.Playing
[746.9654083251953, 745.2017068862915, 801.4237284660339, 685.0425004959106, 727.6479005813599] 60 0



Bitrate
727.6479005813599
State.Playing
Same bitrate ++ !!
[745.2017068862915, 801.4237284660339, 685.0425004959106, 727.6479005813599, 727.6479005813599] 61 0



Bitrate
745.9369897842407
State.Playing
[801.4237284660339, 685.0425004959106, 727.6479005813599, 727.6479005813599, 745.9369897842407] 62 0



Bitrate
747.9046583175659
State.Playing
[685.0425004959106, 727.6479005813599, 727.6479005813599, 745.9369897842407, 747.9046583175659] 63 0



Bitrate
719.2386984825134
State.Playing
[727.6479005813599, 727.6479005813599, 745.9369897842407, 747.9046583175659, 719.2386984825134] 64 0



Bitrate
745.5745339393616
State.Playing
[727.6479005813599, 745.9369897842407, 747.9046583175659, 719.2386984825134, 745.5745339393616] 65 0



Bitrate
745.5745339393616
State.Playing
Same bitrate ++ !!
[745.9369897842407, 747.9046583175659, 719.238698482513




Bitrate
591.1380648612976
State.Playing
[744.005024433136, 743.8178658485413, 745.1213598251343, 883.7758302688599, 591.1380648612976] 117 0



Bitrate
744.924783706665
State.Playing
[743.8178658485413, 745.1213598251343, 883.7758302688599, 591.1380648612976, 744.924783706665] 118 0



Bitrate
745.3810572624207
State.Playing
[745.1213598251343, 883.7758302688599, 591.1380648612976, 744.924783706665, 745.3810572624207] 119 0



Bitrate
747.1018433570862
State.Playing
[883.7758302688599, 591.1380648612976, 744.924783706665, 745.3810572624207, 747.1018433570862] 120 0




####################
737.7826866383353 120 0
####################

Switch Version
Done
Bitrate
0.0
State.Playing
Just starting!
[] 0 0



Bitrate
929.1836619377136
State.Playing
[929.1836619377136] 1 0



Bitrate
745.147168636322
State.Playing
[929.1836619377136, 745.147168636322] 2 0



Bitrate
724.5192527770996
State.Playing
[929.1836619377136, 745.147168636322, 724.5192527770996] 3 0



Bitrate
767.2312259674072
St




Bitrate
823.4993815422058
State.Playing
[746.2313771247864, 862.2685670852661, 672.0252633094788, 606.9785952568054, 823.4993815422058] 49 4



Bitrate
749.4736909866333
State.Playing
[862.2685670852661, 672.0252633094788, 606.9785952568054, 823.4993815422058, 749.4736909866333] 50 4



Bitrate
863.121509552002
State.Playing
[672.0252633094788, 606.9785952568054, 823.4993815422058, 749.4736909866333, 863.121509552002] 51 4



Bitrate
650.3985524177551
State.Playing
[606.9785952568054, 823.4993815422058, 749.4736909866333, 863.121509552002, 650.3985524177551] 52 4



Bitrate
711.0057473182678
State.Playing
[823.4993815422058, 749.4736909866333, 863.121509552002, 650.3985524177551, 711.0057473182678] 53 4



Bitrate
711.0057473182678
State.Playing
Same bitrate ++ !!
[749.4736909866333, 863.121509552002, 650.3985524177551, 711.0057473182678, 711.0057473182678] 54 4



Bitrate
275.8348882198334
State.Playing
[863.121509552002, 650.3985524177551, 711.0057473182678, 711.0057473182678, 275

Process Process-5:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-d232557e96a8>", line 94, in consume_stream_proc
    time.sleep(1)
KeyboardInterrupt
